# Deploy

In [17]:
#library to read and import file from drive
import zipfile

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#images
import cv2 as cv
import matplotlib.pyplot as plt
import seaborn_image as isns
import skimage.exposure

import numpy as np
from numpy.random import default_rng
import random
import tensorflow as tf
import pandas as pd

from tensorflow import keras

In [18]:
from utility import split_patches, scale_pixels, resize, crop, predict_image, localize_anom, calc_image_diff

### Load pretrained model

In [19]:
autoencoder = keras.models.load_model("../Models/autoencoderFC")

### Save predicted images 

In [20]:
dir = "../Data/Inference"

In [21]:
imgs_anom, imgs_name, imgs_score, imgs_valid = [], [], [], []

for img in os.listdir(dir):
    image = cv.imread((dir + '/' + img), cv.IMREAD_GRAYSCALE)
    anom, img_anom, score = predict_image(image, autoencoder)
    imgs_name.append(img)
    imgs_anom.append(anom)
    imgs_score.append(score)
    if score < 0.75:
        imgs_valid.append("Non valid prediction")
    else:
        imgs_valid.append("Valid prediction")
        
    cv.imwrite(dir + "/pred_" + img , img_anom*255)

### Save summary csv

In [29]:
df = {"name_img": imgs_name,
      "anomaly": imgs_anom,
      "reconstruction_score": imgs_score,
      "threshold": imgs_valid}

In [30]:
df = pd.DataFrame(df, index=None)
df.to_csv(dir + "/summary.csv")
df.head()

,name_img,anomaly,reconstruction_score,threshold
0,NOK_2.jpg,True,0.965926,Valid prediction
1,NOK_7.jpg,True,0.974957,Valid prediction
2,OK_1.jpg,False,0.976762,Valid prediction
